# **Identifying Challenges Faced by Developers in Scientific Workflow Management Systems using BERTopic**

This notebook uses BERTopic to identify challenges faced by developers in scientific workflow management systems using Stack Overflow posts and GitHub issues. The dataset used in this notebook is available at https://figshare.com/projects/SWsChallengesbySOandGitHub/172476.

## Installing Dependencies

In [9]:
%pip install -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


## Define functions to preprocess data

In [10]:
import pandas as pd
import re
import nltk
from bs4 import BeautifulSoup
from bertopic import BERTopic

# Download stopwords if not already available
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def clean_text(text):
    # Remove HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()

    # Remove non-alphanumeric characters and convert to lowercase
    text = re.sub(r'[^A-Za-z]', ' ', text.lower())

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def remove_stopwords_and_stem(text):
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    return " ".join(stemmer.stem(word) for word in text.split() if word not in stop_words)

def train_bertopic(data):
    model = BERTopic(language="english", calculate_probabilities=True)
    topics, probabilities = model.fit_transform(data)
    return model, topics, probabilities

[nltk_data] Downloading package stopwords to /home/dev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Import and Preprocess Data

In [11]:
# Import the data
try:
    new_df = pd.read_csv('Dataset/StackOverflowPostsDataset.csv')
    new_df["merged"] = new_df[["Body", "Title", "Tags"]].apply("-".join, axis=1)
except FileNotFoundError:
    print("Dataset file not found. Please provide the correct file path.")
    exit(1)

# Preprocess the data
new_df["merged"] = new_df["merged"].apply(clean_text)
new_df["processed"] = new_df["merged"].apply(remove_stopwords_and_stem)

# Save the preprocessed data
new_df.to_csv('Results/ConcatenatedDatasetSO.csv', index=False)

## Building and training the model

Instantiate the model and train it on the data. The model will automatically select the best topic based on the topic coherence. The higher the topic coherence, the better.

In [12]:
# Train BERTopic on processed data
data = new_df["processed"].values.tolist()
model, topics, probabilities = train_bertopic(data)

# Get topics and their top words
topics_df = model.get_topic_freq()
topics_df.head()

# Save the BERTopic model
model.save("Results/BERTopicModelSO")

## Extracting Topics
After fitting the model, we can extract the topics from the model. This will return the topics with their corresponding IDs, the dominant topic per sentence, and the frequency of each topic.

In [13]:
freq = model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1735,-1_tabl_teradata_date_select,"[tabl, teradata, date, select, valu, id, colum...",[rack brain entir afternoon feel like miss som...
1,0,1307,0_sampl_rule_snakemak_input,"[sampl, rule, snakemak, input, output, fastq, ...",[pipelin creat temporarili file delet later ru...
2,1,431,1_eclips_org_java_intern,"[eclips, org, java, intern, core, kepler, ui, ...",[follow instruct page http wiki eclips org faq...
3,2,372,2_monetdb_databas_tabl_queri,"[monetdb, databas, tabl, queri, monetdblit, da...",[tri point monetdb e regular monetdb databas d...
4,3,285,3_luigi_task_self_def,"[luigi, task, self, def, worker, schedul, run,...",[write luigi workflow tri debug task order nee...


-1 refers to all outliers that BERTopic was not able to assign a topic to. Next, look at the most frequent topics and their words to determine what the topic is about.

In [14]:
model.get_topic(0)  # Select the most frequent topic

[('sampl', 0.026440643494849973),
 ('rule', 0.025838593352225478),
 ('snakemak', 0.024633174593775654),
 ('input', 0.022920482737443897),
 ('output', 0.021872850721014678),
 ('fastq', 0.018508847252945056),
 ('file', 0.01773702947206992),
 ('wildcard', 0.016291796982137407),
 ('gz', 0.015133650621707977),
 ('config', 0.015061385038115242)]

## Assess trained model

In [15]:
# assess predicted topics for first 10 posts
model.topics_[:10]

[-1, -1, 46, 9, 9, 41, 57, -1, 19, -1]

## Visualize Topics
After having trained our model, we can visualize the topics that were generated in a way very similar to LDAvis.

In [16]:
model.visualize_topics()